## MASKED ANN

| Model / Iteration  | Dropout | L2 Reg. | BatchNorm | EarlyStopping | LR Scheduler      | Scaling                       | Physics Features | Notes                             |
| ------------------ | ------- | ------- | --------- | ------------- | ----------------- | ----------------------------- | ---------------- | --------------------------------- |
| **Masked ANN**     | 0.2     | 1e-4    | Yes       | Yes           | ReduceLROnPlateau | StdScaler (X), Ringing→MinMax | Yes              | Masked per-target + physics       |


In [1]:
# ==================== MASKED ANN (All-Best Techniques) ====================
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, regularizers
import matplotlib.pyplot as plt

# ============== SETTINGS ==============
SEED = 42
UNSEEN_PART = 'C2M0040120D'
BASE_DIR = "iteration_masked"
os.makedirs(f"{BASE_DIR}/r2_rmse_tables", exist_ok=True)
os.makedirs(f"{BASE_DIR}/train_val_loss_curves", exist_ok=True)
os.makedirs(f"{BASE_DIR}/models", exist_ok=True)

# ============== LOAD DATA ==============
df = pd.read_csv(
    r"C:\Users\pc\Desktop\Neural_Network_Models\merged_train_5_MOSFETs_10percent_balanced.csv"
)

TARGET_COLUMNS = [
    'voltage_rise_time_pulse1', 'voltage_rise_time_pulse2',
    'voltage_fall_time_pulse1', 'voltage_fall_time_pulse2',
    'current_rise_time_pulse1', 'current_rise_time_pulse2',
    'current_fall_time_pulse1', 'current_fall_time_pulse2',
    'overshoot_pulse_1', 'overshoot_pulse_2',
    'undershoot_pulse_1', 'undershoot_pulse_2',
    'ringing_frequency_MHz'
]
DROP_COLUMNS = ['DeviceID', 'MOSFET', 'Part_Number']

# ============== SPLIT SEEN/UNSEEN DEVICES ==============
seen_parts = [p for p in df['Part_Number'].unique().tolist() if p != UNSEEN_PART]
train_df = df[df['Part_Number'].isin(seen_parts)].copy()
test_df  = df[df['Part_Number'] == UNSEEN_PART].copy()

# ============== PHYSICS FEATURES ==============
def compute_physics_features(row):
    L_eq = row[['Ls4','Ls5','Ls6','Ls7','Ls8','Ls9','Ls10','Ls11']].sum()
    C_eq = row.get("Coss", 1e-12)
    f_res = 1/(2*np.pi*np.sqrt(L_eq*C_eq))/1e6 if L_eq>0 and C_eq>0 else 0
    overshoot_est = row.get("VDS_max",0) - row.get("Vbus",0)
    undershoot_est = - row.get("VGS_th_min",0)
    dVdt_est = row.get("VDS_max",0)/row.get("Tp1",1e-9)
    dIdt_est = row.get("ID_max_25C",0)/row.get("Tp1",1e-9)
    return pd.Series([f_res, overshoot_est, undershoot_est, dVdt_est, dIdt_est])

for df_ in [train_df, test_df]:
    df_[['f_resonance','overshoot_est','undershoot_est','dVdt_est','dIdt_est']] = \
        df_.apply(compute_physics_features, axis=1)

physics_features = ['f_resonance','overshoot_est','undershoot_est','dVdt_est','dIdt_est']
INPUT_COLUMNS = [c for c in df.columns if c not in TARGET_COLUMNS + DROP_COLUMNS] + physics_features

# ============== SCALE INPUTS ==============
input_scaler = StandardScaler()
input_scaler.fit(pd.concat([train_df[INPUT_COLUMNS], test_df[INPUT_COLUMNS]]))
X_train_all = input_scaler.transform(train_df[INPUT_COLUMNS])
X_test_all  = input_scaler.transform(test_df[INPUT_COLUMNS])

# ============== SCALE OUTPUTS (per target) ==============
output_scalers, y_train_scaled, y_test_scaled = {}, [], []
for col in TARGET_COLUMNS:
    scaler = MinMaxScaler() if col == 'ringing_frequency_MHz' else StandardScaler()
    y_train_scaled.append(scaler.fit_transform(train_df[[col]]).flatten())
    y_test_scaled.append(scaler.transform(test_df[[col]]).flatten())
    output_scalers[col] = scaler
y_train_scaled = np.array(y_train_scaled).T
y_test_scaled  = np.array(y_test_scaled).T

# ============== SPLIT TRAIN/VAL ==============
Xtr, Xval, ytr, yval = train_test_split(
    X_train_all, y_train_scaled, test_size=0.15, random_state=SEED
)

# ============== FEATURE MASKS (toy example: select subsets) ==============
# In practice, you’d compute mutual info or importance scores per target.
# Here we just group physics features + a few indices per target for demonstration.
feature_masks = {
    'voltage_rise_time_pulse1': physics_features[:3],
    'voltage_fall_time_pulse1': physics_features[2:],
    'current_rise_time_pulse1': physics_features[:2],
    'current_fall_time_pulse1': physics_features[1:],
    'overshoot_pulse_1': physics_features,
    'undershoot_pulse_1': physics_features,
    'ringing_frequency_MHz': physics_features
}

# Map feature names to indices
mask_indices = {col: [INPUT_COLUMNS.index(f) for f in feats if f in INPUT_COLUMNS] 
                for col, feats in feature_masks.items()}

# ============== BUILD MASKED ANN ==============
def build_masked_ann(input_dim, output_dim, masks, l2_reg=1e-4, dropout_rate=0.2):
    inp = layers.Input(shape=(input_dim,), name="global_inputs")

    # Shared backbone
    x = layers.Dense(128, kernel_regularizer=regularizers.l2(l2_reg))(inp)
    x = layers.BatchNormalization()(x); x = layers.ReLU()(x)
    x = layers.Dropout(dropout_rate)(x)

    x = layers.Dense(64, kernel_regularizer=regularizers.l2(l2_reg))(x)
    x = layers.BatchNormalization()(x); x = layers.ReLU()(x)
    x = layers.Dropout(dropout_rate)(x)

    outputs = []
    for i, col in enumerate(TARGET_COLUMNS):
        # Mask input branch
        if col in masks and len(masks[col]) > 0:
            masked = layers.Lambda(lambda z, idx=masks[col]: tf.gather(z, idx, axis=1))(inp)
            m = layers.Dense(32, activation="relu")(masked)
        else:
            m = layers.Dense(16, activation="relu")(inp)

        # Combine backbone + masked branch
        h = layers.Concatenate()([x, m])
        h = layers.Dense(64, activation="relu")(h)
        h = layers.Dropout(dropout_rate)(h)
        h = layers.Dense(32, activation="relu")(h)
        out = layers.Dense(1, activation="linear", name=col)(h)
        outputs.append(out)

    model = models.Model(inputs=inp, outputs=outputs)
    return model

model = build_masked_ann(Xtr.shape[1], len(TARGET_COLUMNS), mask_indices)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
    loss=["mse"]*len(TARGET_COLUMNS),
    metrics=["mae"]*len(TARGET_COLUMNS)
)

early_stop = callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
reduce_lr  = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=5)

history = model.fit(
    Xtr, [ytr[:,i] for i in range(len(TARGET_COLUMNS))],
    validation_data=(Xval,[yval[:,i] for i in range(len(TARGET_COLUMNS))]),
    epochs=120, batch_size=256, callbacks=[early_stop, reduce_lr], verbose=1
)

model.save(f"{BASE_DIR}/models/masked_ann.h5")

# ============== LOSS CURVE ==============
plt.figure()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel("Epoch"); plt.ylabel("MSE Loss")
plt.title("Train vs Validation Loss")
plt.legend(); plt.savefig(f"{BASE_DIR}/train_val_loss_curves/loss.png"); plt.close()

# ============== EVALUATION ==============
def evaluate_and_save(X, y_scaled, name, positive_only=False):
    y_pred_scaled_list = model.predict(X)
    results = []
    for i, col in enumerate(TARGET_COLUMNS):
        y_true = output_scalers[col].inverse_transform(y_scaled[:,i].reshape(-1,1)).flatten()
        y_pred = output_scalers[col].inverse_transform(y_pred_scaled_list[i].reshape(-1,1)).flatten()
        r2 = r2_score(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        results.append((col, r2, rmse))
    df_results = pd.DataFrame(results, columns=["Target","R2","RMSE"]).sort_values("R2",ascending=False)
    if positive_only:
        df_results = df_results[df_results["R2"]>0]
    print(f"\n{name}:\n",df_results)
    df_results.to_csv(f"{BASE_DIR}/r2_rmse_tables/{name}.csv", index=False)
    return df_results

evaluate_and_save(Xtr,ytr,"train")
evaluate_and_save(Xval,yval,"val")
evaluate_and_save(X_train_all,y_train_scaled,"test")
evaluate_and_save(X_test_all,y_test_scaled,"unseen",positive_only=True)



Epoch 1/120
29/29 ━━━━━━━━━━━━━━━━━━━━ 13s 52ms/step - current_fall_time_pulse1_loss: 2.4923 - current_fall_time_pulse1_mae: 1.3218 - current_fall_time_pulse2_loss: 1.8085 - current_fall_time_pulse2_mae: 1.0985 - current_rise_time_pulse1_loss: 0.7585 - current_rise_time_pulse1_mae: 0.6648 - current_rise_time_pulse2_loss: 0.6667 - current_rise_time_pulse2_mae: 0.6633 - loss: 13.4141 - overshoot_pulse_1_loss: 0.8979 - overshoot_pulse_1_mae: 0.7326 - overshoot_pulse_2_loss: 0.8456 - overshoot_pulse_2_mae: 0.7184 - ringing_frequency_MHz_loss: 0.3998 - ringing_frequency_MHz_mae: 0.4908 - undershoot_pulse_1_loss: 1.0898 - undershoot_pulse_1_mae: 0.8148 - undershoot_pulse_2_loss: 0.9496 - undershoot_pulse_2_mae: 0.7520 - voltage_fall_time_pulse1_loss: 0.6584 - voltage_fall_time_pulse1_mae: 0.6404 - voltage_fall_time_pulse2_loss: 1.0339 - voltage_fall_time_pulse2_mae: 0.7424 - voltage_rise_time_pulse1_loss: 0.8045 - voltage_rise_time_pulse1_mae: 0.7083 - voltage_rise_time_pulse2_loss: 0.9906 

230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step

train:
                       Target        R2          RMSE
7   current_fall_time_pulse2  0.983588  1.498985e-09
3   voltage_fall_time_pulse2  0.982325  9.098929e-10
2   voltage_fall_time_pulse1  0.980108  9.663848e-10
11        undershoot_pulse_2  0.971810  2.329953e+00
10        undershoot_pulse_1  0.965475  2.573389e+00
6   current_fall_time_pulse1  0.961037  2.317901e-09
0   voltage_rise_time_pulse1  0.960639  7.994149e-10
12     ringing_frequency_MHz  0.934445  8.412369e+00
8          overshoot_pulse_1  0.921004  3.236195e+00
9          overshoot_pulse_2  0.918215  7.077078e+00
4   current_rise_time_pulse1  0.915316  1.420996e-08
5   current_rise_time_pulse2  0.880416  8.528745e-09
1   voltage_rise_time_pulse2  0.879096  1.335990e-09
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

val:
                       Target        R2          RMSE
7   current_fall_time_pulse2  0.983789  1.515622e-09
3   voltage_fall_time_pulse2  0.982621  9.295721e-10
2  

,Target,R2,RMSE
2,voltage_fall_time_pulse1,0.875371,8.216623e-10
3,voltage_fall_time_pulse2,0.786670,1.075555e-09
6,current_fall_time_pulse1,0.727609,5.860866e-09
8,overshoot_pulse_1,0.641765,7.603426e+00
9,overshoot_pulse_2,0.577482,1.291390e+01
11,undershoot_pulse_2,0.567099,6.266214e+00
10,undershoot_pulse_1,0.534943,6.541878e+00
7,current_fall_time_pulse2,0.462694,8.175133e-09
0,voltage_rise_time_pulse1,0.457260,1.284393e-09
1,voltage_rise_time_pulse2,0.061485,4.267736e-09


In [2]:
# ==================== MASKED ANN (All-Best Techniques) ====================
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, regularizers
import matplotlib.pyplot as plt

# ============== SETTINGS ==============
SEED = 42
UNSEEN_PART = 'C2M0040120D'
BASE_DIR = "iteration_masked"
os.makedirs(f"{BASE_DIR}/r2_rmse_tables", exist_ok=True)
os.makedirs(f"{BASE_DIR}/train_val_loss_curves", exist_ok=True)
os.makedirs(f"{BASE_DIR}/models", exist_ok=True)

# ============== LOAD DATA ==============
df = pd.read_csv(
    r"C:\Users\pc\Desktop\Neural_Network_Models\merged_train_5_MOSFETs_25percent_balanced.csv"
)

TARGET_COLUMNS = [
    'voltage_rise_time_pulse1', 'voltage_rise_time_pulse2',
    'voltage_fall_time_pulse1', 'voltage_fall_time_pulse2',
    'current_rise_time_pulse1', 'current_rise_time_pulse2',
    'current_fall_time_pulse1', 'current_fall_time_pulse2',
    'overshoot_pulse_1', 'overshoot_pulse_2',
    'undershoot_pulse_1', 'undershoot_pulse_2',
    'ringing_frequency_MHz'
]
DROP_COLUMNS = ['DeviceID', 'MOSFET', 'Part_Number']

# ============== SPLIT SEEN/UNSEEN DEVICES ==============
seen_parts = [p for p in df['Part_Number'].unique().tolist() if p != UNSEEN_PART]
train_df = df[df['Part_Number'].isin(seen_parts)].copy()
test_df  = df[df['Part_Number'] == UNSEEN_PART].copy()

# ============== PHYSICS FEATURES ==============
def compute_physics_features(row):
    L_eq = row[['Ls4','Ls5','Ls6','Ls7','Ls8','Ls9','Ls10','Ls11']].sum()
    C_eq = row.get("Coss", 1e-12)
    f_res = 1/(2*np.pi*np.sqrt(L_eq*C_eq))/1e6 if L_eq>0 and C_eq>0 else 0
    overshoot_est = row.get("VDS_max",0) - row.get("Vbus",0)
    undershoot_est = - row.get("VGS_th_min",0)
    dVdt_est = row.get("VDS_max",0)/row.get("Tp1",1e-9)
    dIdt_est = row.get("ID_max_25C",0)/row.get("Tp1",1e-9)
    return pd.Series([f_res, overshoot_est, undershoot_est, dVdt_est, dIdt_est])

for df_ in [train_df, test_df]:
    df_[['f_resonance','overshoot_est','undershoot_est','dVdt_est','dIdt_est']] = \
        df_.apply(compute_physics_features, axis=1)

physics_features = ['f_resonance','overshoot_est','undershoot_est','dVdt_est','dIdt_est']
INPUT_COLUMNS = [c for c in df.columns if c not in TARGET_COLUMNS + DROP_COLUMNS] + physics_features

# ============== SCALE INPUTS ==============
input_scaler = StandardScaler()
input_scaler.fit(pd.concat([train_df[INPUT_COLUMNS], test_df[INPUT_COLUMNS]]))
X_train_all = input_scaler.transform(train_df[INPUT_COLUMNS])
X_test_all  = input_scaler.transform(test_df[INPUT_COLUMNS])

# ============== SCALE OUTPUTS (per target) ==============
output_scalers, y_train_scaled, y_test_scaled = {}, [], []
for col in TARGET_COLUMNS:
    scaler = MinMaxScaler() if col == 'ringing_frequency_MHz' else StandardScaler()
    y_train_scaled.append(scaler.fit_transform(train_df[[col]]).flatten())
    y_test_scaled.append(scaler.transform(test_df[[col]]).flatten())
    output_scalers[col] = scaler
y_train_scaled = np.array(y_train_scaled).T
y_test_scaled  = np.array(y_test_scaled).T

# ============== SPLIT TRAIN/VAL ==============
Xtr, Xval, ytr, yval = train_test_split(
    X_train_all, y_train_scaled, test_size=0.15, random_state=SEED
)

# ============== FEATURE MASKS (toy example: select subsets) ==============
# In practice, you’d compute mutual info or importance scores per target.
# Here we just group physics features + a few indices per target for demonstration.
feature_masks = {
    'voltage_rise_time_pulse1': physics_features[:3],
    'voltage_fall_time_pulse1': physics_features[2:],
    'current_rise_time_pulse1': physics_features[:2],
    'current_fall_time_pulse1': physics_features[1:],
    'overshoot_pulse_1': physics_features,
    'undershoot_pulse_1': physics_features,
    'ringing_frequency_MHz': physics_features
}

# Map feature names to indices
mask_indices = {col: [INPUT_COLUMNS.index(f) for f in feats if f in INPUT_COLUMNS] 
                for col, feats in feature_masks.items()}

# ============== BUILD MASKED ANN ==============
def build_masked_ann(input_dim, output_dim, masks, l2_reg=1e-4, dropout_rate=0.2):
    inp = layers.Input(shape=(input_dim,), name="global_inputs")

    # Shared backbone
    x = layers.Dense(128, kernel_regularizer=regularizers.l2(l2_reg))(inp)
    x = layers.BatchNormalization()(x); x = layers.ReLU()(x)
    x = layers.Dropout(dropout_rate)(x)

    x = layers.Dense(64, kernel_regularizer=regularizers.l2(l2_reg))(x)
    x = layers.BatchNormalization()(x); x = layers.ReLU()(x)
    x = layers.Dropout(dropout_rate)(x)

    outputs = []
    for i, col in enumerate(TARGET_COLUMNS):
        # Mask input branch
        if col in masks and len(masks[col]) > 0:
            masked = layers.Lambda(lambda z, idx=masks[col]: tf.gather(z, idx, axis=1))(inp)
            m = layers.Dense(32, activation="relu")(masked)
        else:
            m = layers.Dense(16, activation="relu")(inp)

        # Combine backbone + masked branch
        h = layers.Concatenate()([x, m])
        h = layers.Dense(64, activation="relu")(h)
        h = layers.Dropout(dropout_rate)(h)
        h = layers.Dense(32, activation="relu")(h)
        out = layers.Dense(1, activation="linear", name=col)(h)
        outputs.append(out)

    model = models.Model(inputs=inp, outputs=outputs)
    return model

model = build_masked_ann(Xtr.shape[1], len(TARGET_COLUMNS), mask_indices)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
    loss=["mse"]*len(TARGET_COLUMNS),
    metrics=["mae"]*len(TARGET_COLUMNS)
)

early_stop = callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
reduce_lr  = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=5)

history = model.fit(
    Xtr, [ytr[:,i] for i in range(len(TARGET_COLUMNS))],
    validation_data=(Xval,[yval[:,i] for i in range(len(TARGET_COLUMNS))]),
    epochs=120, batch_size=256, callbacks=[early_stop, reduce_lr], verbose=1
)

model.save(f"{BASE_DIR}/models/masked_ann.h5")

# ============== LOSS CURVE ==============
plt.figure()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel("Epoch"); plt.ylabel("MSE Loss")
plt.title("Train vs Validation Loss")
plt.legend(); plt.savefig(f"{BASE_DIR}/train_val_loss_curves/loss.png"); plt.close()

# ============== EVALUATION ==============
def evaluate_and_save(X, y_scaled, name, positive_only=False):
    y_pred_scaled_list = model.predict(X)
    results = []
    for i, col in enumerate(TARGET_COLUMNS):
        y_true = output_scalers[col].inverse_transform(y_scaled[:,i].reshape(-1,1)).flatten()
        y_pred = output_scalers[col].inverse_transform(y_pred_scaled_list[i].reshape(-1,1)).flatten()
        r2 = r2_score(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        results.append((col, r2, rmse))
    df_results = pd.DataFrame(results, columns=["Target","R2","RMSE"]).sort_values("R2",ascending=False)
    if positive_only:
        df_results = df_results[df_results["R2"]>0]
    print(f"\n{name}:\n",df_results)
    df_results.to_csv(f"{BASE_DIR}/r2_rmse_tables/{name}.csv", index=False)
    return df_results

evaluate_and_save(Xtr,ytr,"train")
evaluate_and_save(Xval,yval,"val")
evaluate_and_save(X_train_all,y_train_scaled,"test")
evaluate_and_save(X_test_all,y_test_scaled,"unseen",positive_only=True)


Epoch 1/120
287/287 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - current_fall_time_pulse1_loss: 0.3948 - current_fall_time_pulse1_mae: 0.4612 - current_fall_time_pulse2_loss: 0.5277 - current_fall_time_pulse2_mae: 0.5331 - current_rise_time_pulse1_loss: 0.3894 - current_rise_time_pulse1_mae: 0.4510 - current_rise_time_pulse2_loss: 0.4073 - current_rise_time_pulse2_mae: 0.4990 - loss: 5.8541 - overshoot_pulse_1_loss: 0.6063 - overshoot_pulse_1_mae: 0.5823 - overshoot_pulse_2_loss: 0.8693 - overshoot_pulse_2_mae: 0.6510 - ringing_frequency_MHz_loss: 0.1603 - ringing_frequency_MHz_mae: 0.2705 - undershoot_pulse_1_loss: 0.4639 - undershoot_pulse_1_mae: 0.5077 - undershoot_pulse_2_loss: 0.4904 - undershoot_pulse_2_mae: 0.5056 - voltage_fall_time_pulse1_loss: 0.3506 - voltage_fall_time_pulse1_mae: 0.3980 - voltage_fall_time_pulse2_loss: 0.3097 - voltage_fall_time_pulse2_mae: 0.3725 - voltage_rise_time_pulse1_loss: 0.3491 - voltage_rise_time_pulse1_mae: 0.4222 - voltage_rise_time_pulse2_loss: 0.5213 

2294/2294 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step

train:
                       Target        R2          RMSE
2   voltage_fall_time_pulse1  0.979575  9.822972e-10
7   current_fall_time_pulse2  0.969697  2.047718e-09
0   voltage_rise_time_pulse1  0.965786  7.494035e-10
6   current_fall_time_pulse1  0.964496  2.223276e-09
3   voltage_fall_time_pulse2  0.963984  1.302844e-09
11        undershoot_pulse_2  0.955648  2.887048e+00
12     ringing_frequency_MHz  0.946555  7.634188e+00
8          overshoot_pulse_1  0.931755  3.195004e+00
10        undershoot_pulse_1  0.931231  3.592969e+00
9          overshoot_pulse_2  0.915597  7.347573e+00
4   current_rise_time_pulse1  0.913262  1.447199e-08
1   voltage_rise_time_pulse2  0.898962  1.280028e-09
5   current_rise_time_pulse2  0.896829  8.067454e-09
405/405 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

val:
                       Target        R2          RMSE
2   voltage_fall_time_pulse1  0.979700  9.745329e-10
7   current_fall_time_pulse2  0.969667  2.031485e-09

,Target,R2,RMSE
7,current_fall_time_pulse2,0.904974,3.437031e-09
2,voltage_fall_time_pulse1,0.877816,8.137873e-10
6,current_fall_time_pulse1,0.741728,5.705518e-09
0,voltage_rise_time_pulse1,0.719622,9.117878e-10
11,undershoot_pulse_2,0.556933,6.485151e+00
8,overshoot_pulse_1,0.544348,9.128501e+00
10,undershoot_pulse_1,0.512488,6.849068e+00
1,voltage_rise_time_pulse2,0.289695,3.931928e-09
5,current_rise_time_pulse2,0.056906,1.628100e-08
9,overshoot_pulse_2,0.031048,2.039334e+01
